In [2]:
words = ["cigar", "rebut", "sissy", "humph", "awake", "blush", "focal", "evade", "naval", "serve", "heath", "dwarf", "model", "karma", "stink", "grade", "quiet", "bench", "abate", "feign", "major", "death", "fresh", "crust", "stool", "colon", "abase", "marry", "react", "batty", "pride", "floss", "helix", "croak", "staff", "paper", "unfed", "whelp", "trawl", "outdo", "adobe", "crazy", "sower", "repay", "digit", "crate", "cluck", "spike", "mimic", "pound", "maxim", "linen", "unmet", "flesh", "booby", "forth", "first", "stand", "belly", "ivory", "seedy", "print", "yearn", "drain", "bribe", "stout", "panel", "crass", "flume", "offal", "agree", "error", "swirl", "argue", "bleed", "delta", "flick", "totem", "wooer", "front", "shrub", "parry", "biome", "lapel", "start", "greet", "goner", "golem", "lusty", "loopy", "round", "audit", "lying", "gamma", "labor", "islet", "civic", "forge", "corny", "moult", "basic", "salad", "agate", "spicy", "spray", "essay", "fjord", "spend", "kebab", "guild", "aback", "motor", "alone", "hatch", "hyper", "thumb", "dowry", "ought", "belch", "dutch", "pilot", "tweed", "comet", "jaunt", "enema", "steed", "abyss", "growl", "fling", "dozen", "boozy", "erode", "world", "gouge", "click", "briar", "great", "altar", "pulpy", "blurt", "coast", "duchy", "groin", "fixer", "group", "rogue", "badly", "smart", "pithy", "gaudy", "chill", "heron", "vodka", "finer", "surer", "radio", "rouge", "perch", "retch", "wrote", "clock", "tilde", "store", "prove", "bring", "solve", "cheat", "grime", "exult", "usher", "epoch", "triad", "break", "rhino", "viral", "conic", "masse", "sonic", "vital", "trace", "using", "peach", "champ", "baton", "brake", "pluck", "craze", "gripe", "weary", "picky", "acute", "ferry", "aside", "tapir", "troll", "unify", "rebus", "boost", "truss", "siege", "tiger", "banal", "slump", "crank", "gorge", "query", "drink", "favor", "abbey", "tangy", "panic", "solar", "shire", "proxy", "point", "robot", "prick", "wince", "crimp", "knoll", "sugar", "whack", "mount", "perky", "could", "wrung", "light", "those", "moist", "shard", "pleat", "aloft", "skill", "elder", "frame", "humor", "pause", "ulcer", "ultra", "robin", "cynic", "aroma", "caulk", "shake", "dodge", "swill", "tacit", "other", "thorn", "trove", "bloke", "vivid", "spill", "chant", "choke", "rupee", "nasty", "mourn", "ahead", "brine", "cloth", "hoard", "sweet", "month", "lapse", "watch", "today", "focus", "smelt", "tease", "cater", "movie", "saute", "allow", "renew", "their", "slosh", "purge", "chest", "depot", "epoxy", "nymph", "found", "shall", "stove", "lowly", "snout", "trope", "fewer", "shawl", "natal", "comma", "foray", "scare", "stair", "black", "squad", "royal", "chunk", "mince", "shame", "cheek", "ample", "flair", "foyer", "cargo", "oxide", "plant", "olive", "inert", "askew", "heist", "shown", "zesty", "trash", "larva", "forgo", "story", "hairy", "train", "homer", "badge", "midst", "canny", "shine", "gecko", "farce", "slung", "tipsy", "metal", "yield", "delve", "being", "scour", "glass", "gamer", "scrap", "money", "hinge", "album", "vouch", "asset", "tiara", "crept", "bayou", "atoll", "manor", "creak", "showy", "phase", "froth", "depth", "gloom", "flood", "trait", "girth", "piety", "goose", "float", "donor", "atone", "primo", "apron", "blown", "cacao", "loser", "input", "gloat", "awful", "brink", "smite", "beady", "rusty", "retro", "droll", "gawky", "hutch", "pinto", "egret", "lilac", "sever", "field", "fluff", "agape", "voice", "stead", "berth", "madam", "night", "bland", "liver", "wedge", "roomy", "wacky", "flock", "angry", "trite", "aphid", "tryst", "midge", "power", "elope", "cinch", "motto", "stomp", "upset", "bluff", "cramp", "quart", "coyly", "youth", "rhyme", "buggy", "alien", "smear", "unfit", "patty", "cling", "glean", "label", "hunky", "khaki", "poker", "gruel", "twice", "twang", "shrug", "treat", "waste", "merit", "woven", "needy", "clown", "irony", "ruder", "gauze", "chief", "onset", "prize", "fungi", "charm", "gully", "inter", "whoop", "taunt", "leery", "class", "theme", "lofty", "tibia", "booze", "alpha", "thyme", "doubt", "parer", "chute", "stick", "trice", "alike", "recap", "saint", "glory", "grate", "admit", "brisk", "soggy", "usurp", "scald", "scorn", "leave", "twine", "sting", "bough", "marsh", "sloth", "dandy", "vigor", "howdy", "enjoy", "valid", "ionic", "equal", "floor", "catch", "spade", "stein", "exist", "quirk", "denim", "grove", "spiel", "mummy", "fault", "foggy", "flout", "carry", "sneak", "libel", "waltz", "aptly", "piney", "inept", "aloud", "photo", "dream", "stale", "unite", "snarl", "baker", "there", "glyph", "pooch", "hippy", "spell", "folly", "louse", "gulch", "vault", "godly", "threw", "fleet", "grave", "inane", "shock", "crave", "spite", "valve", "skimp", "claim", "rainy", "musty", "pique", "daddy", "quasi", "arise", "aging", "valet", "opium", "avert", "stuck", "recut", "mulch", "genre", "plume", "rifle", "count", "incur", "total", "wrest", "mocha", "deter", "study", "lover", "safer", "rivet", "funny", "smoke", "mound", "undue", "sedan", "pagan", "swine", "guile", "gusty", "equip", "tough", "canoe", "chaos", "covet", "human", "udder", "lunch", "blast", "stray", "manga", "melee", "lefty", "quick", "paste", "given", "octet", "risen", "groan", "leaky", "grind", "carve", "loose", "sadly", "spilt", "apple", "slack", "honey", "final", "sheen", "eerie", "minty", "slick", "derby", "wharf", "spelt", "coach", "erupt", "singe", "price", "spawn", "fairy", "jiffy", "filmy", "stack", "chose", "sleep", "ardor", "nanny", "niece", "woozy", "handy", "grace", "ditto", "stank", "cream", "usual", "diode", "valor", "angle", "ninja", "muddy", "chase", "reply", "prone", "spoil", "heart", "shade", "diner", "arson", "onion", "sleet", "dowel", "couch", "palsy", "bowel", "smile", "evoke", "creek", "lance", "eagle", "idiot", "siren", "built", "embed", "award", "dross", "annul", "goody", "frown", "patio", "laden", "humid", "elite", "lymph", "edify", "might", "reset", "visit", "gusto", "purse", "vapor", "crock", "write", "sunny", "loath", "chaff", "slide", "queer", "venom", "stamp", "sorry", "still", "acorn", "aping", "pushy", "tamer", "hater", "mania", "awoke", "brawn", "swift", "exile", "birch", "lucky", "freer", "risky", "ghost", "plier", "lunar", "winch", "snare", "nurse", "house", "borax", "nicer", "lurch", "exalt", "about", "savvy", "toxin", "tunic", "pried", "inlay", "chump", "lanky", "cress", "eater", "elude", "cycle", "kitty", "boule", "moron", "tenet", "place", "lobby", "plush", "vigil", "index", "blink", "clung", "qualm", "croup", "clink", "juicy", "stage", "decay", "nerve", "flier", "shaft", "crook", "clean", "china", "ridge", "vowel", "gnome", "snuck", "icing", "spiny", "rigor", "snail", "flown", "rabid", "prose", "thank", "poppy", "budge", "fiber", "moldy", "dowdy", "kneel", "track", "caddy", "quell", "dumpy", "paler", "swore", "rebar", "scuba", "splat", "flyer", "horny", "mason", "doing", "ozone", "amply", "molar", "ovary", "beset", "queue", "cliff", "magic", "truce", "sport", "fritz", "edict", "twirl", "verse", "llama", "eaten", "range", "whisk", "hovel", "rehab", "macaw", "sigma", "spout", "verve", "sushi", "dying", "fetid", "brain", "buddy", "thump", "scion", "candy", "chord", "basin", "march", "crowd", "arbor", "gayly", "musky", "stain", "dally", "bless", "bravo", "stung", "title", "ruler", "kiosk", "blond", "ennui", "layer", "fluid", "tatty", "score", "cutie", "zebra", "barge", "matey", "bluer", "aider", "shook", "river", "privy", "betel", "frisk", "bongo", "begun", "azure", "weave", "genie", "sound", "glove", "braid", "scope", "wryly", "rover", "assay", "ocean", "bloom", "irate", "later", "woken", "silky", "wreck", "dwelt", "slate", "smack", "solid", "amaze", "hazel", "wrist", "jolly", "globe", "flint", "rouse", "civil", "vista", "relax", "cover", "alive", "beech", "jetty", "bliss", "vocal", "often", "dolly", "eight", "joker", "since", "event", "ensue", "shunt", "diver", "poser", "worst", "sweep", "alley", "creed", "anime", "leafy", "bosom", "dunce", "stare", "pudgy", "waive", "choir", "stood", "spoke", "outgo", "delay", "bilge", "ideal", "clasp", "seize", "hotly", "laugh", "sieve", "block", "meant", "grape", "noose", "hardy", "shied", "drawl", "daisy", "putty", "strut", "burnt", "tulip", "crick", "idyll", "vixen", "furor", "geeky", "cough", "naive", "shoal", "stork", "bathe", "aunty", "check", "prime", "brass", "outer", "furry", "razor", "cigar", "rebut", "sissy", "humph", "awake", "blush", "focal", "evade", "naval", "serve", "heath", "dwarf", "model", "karma", "stink", "grade", "quiet", "bench", "abate", "feign", "major", "death", "fresh", "crust", "stool", "colon", "abase", "marry", "react", "batty", "pride", "floss", "helix", "croak", "staff", "paper", "unfed", "whelp", "trawl", "outdo", "adobe", "crazy", "sower", "repay", "digit", "crate", "cluck", "spike", "mimic", "pound", "maxim", "linen", "unmet", "flesh", "booby", "forth", "first", "stand", "belly", "ivory", "seedy", "print", "yearn", "drain", "bribe", "stout", "panel", "crass", "flume", "offal", "agree", "error", "swirl", "argue", "bleed", "delta", "flick", "totem", "wooer", "front", "shrub", "parry", "biome", "lapel", "start", "greet", "goner", "golem", "lusty", "loopy", "round", "audit", "lying", "gamma", "labor", "islet", "civic", "forge", "corny", "moult", "basic", "salad", "agate", "spicy", "spray", "essay", "fjord", "spend", "kebab", "guild", "aback", "motor", "alone", "hatch", "hyper", "thumb", "dowry", "ought", "belch", "dutch", "pilot", "tweed", "comet", "jaunt", "enema", "steed", "abyss", "growl", "fling", "dozen", "boozy", "erode", "world", "gouge", "click", "briar", "great", "altar", "pulpy", "blurt", "coast", "duchy", "groin", "fixer", "group", "rogue", "badly", "smart", "pithy", "gaudy", "chill", "heron", "vodka", "finer", "surer", "radio", "rouge", "perch", "retch", "wrote", "clock", "tilde", "store", "prove", "bring", "solve", "cheat", "grime", "exult", "usher", "epoch", "triad", "break", "rhino", "viral", "conic", "masse", "sonic", "vital", "trace", "using", "peach", "champ", "baton", "brake", "pluck", "craze", "gripe", "weary", "picky", "acute", "ferry", "aside", "tapir", "troll", "unify", "rebus", "boost", "truss", "siege", "tiger", "banal", "slump", "crank", "gorge", "query", "drink", "favor", "abbey", "tangy", "panic", "solar", "shire", "proxy", "point", "robot", "prick", "wince", "crimp", "knoll", "sugar", "whack", "mount", "perky", "could", "wrung", "light", "those", "moist", "shard", "pleat", "aloft", "skill", "elder", "frame", "humor", "pause", "ulcer", "ultra", "robin", "cynic", "aroma", "caulk", "shake", "dodge", "swill", "tacit", "other", "thorn", "trove", "bloke", "vivid", "spill", "chant", "choke", "rupee", "nasty", "mourn", "ahead", "brine", "cloth", "hoard", "sweet", "month", "lapse", "watch", "today", "focus", "smelt", "tease", "cater", "movie", "saute", "allow", "renew", "their", "slosh", "purge", "chest", "depot", "epoxy", "nymph", "found", "shall", "stove", "lowly", "snout", "trope", "fewer", "shawl", "natal", "comma", "foray", "scare", "stair", "black", "squad", "royal", "chunk", "mince", "shame", "cheek", "ample", "flair", "foyer", "cargo", "oxide", "plant", "olive", "inert", "askew", "heist", "shown", "zesty", "trash", "larva", "forgo", "story", "hairy", "train", "homer", "badge", "midst", "canny", "shine", "gecko", "farce", "slung", "tipsy", "metal", "yield", "delve", "being", "scour", "glass", "gamer", "scrap", "money", "hinge", "album", "vouch", "asset", "tiara", "crept", "bayou", "atoll", "manor", "creak", "showy", "phase", "froth", "depth", "gloom", "flood", "trait", "girth", "piety", "goose", "float", "donor", "atone", "primo", "apron", "blown", "cacao", "loser", "input", "gloat", "awful", "brink", "smite", "beady", "rusty", "retro", "droll", "gawky", "hutch", "pinto", "egret", "lilac", "sever", "field", "fluff", "agape", "voice", "stead", "berth", "madam", "night", "bland", "liver", "wedge", "roomy", "wacky", "flock", "angry", "trite", "aphid", "tryst", "midge", "power", "elope", "cinch", "motto", "stomp", "upset", "bluff", "cramp", "quart", "coyly", "youth", "rhyme", "buggy", "alien", "smear", "unfit", "patty", "cling", "glean", "label", "hunky", "khaki", "poker", "gruel", "twice", "twang", "shrug", "treat", "waste", "merit", "woven", "needy", "clown", "irony", "ruder", "gauze", "chief", "onset", "prize", "fungi", "charm", "gully", "inter", "whoop", "taunt", "leery", "class", "theme", "lofty", "tibia", "booze", "alpha", "thyme", "doubt", "parer", "chute", "stick", "trice", "alike", "recap", "saint", "glory", "grate", "admit", "brisk", "soggy", "usurp", "scald", "scorn", "leave", "twine", "sting", "bough", "marsh", "sloth", "dandy", "vigor", "howdy", "enjoy", "valid", "ionic", "equal", "floor", "catch", "spade", "stein", "exist", "quirk", "denim", "grove", "spiel", "mummy", "fault", "foggy", "flout", "carry", "sneak", "libel", "waltz", "aptly", "piney", "inept", "aloud", "photo", "dream", "stale", "unite", "snarl", "baker", "there", "glyph", "pooch", "hippy", "spell", "folly", "louse", "gulch", "vault", "godly", "threw", "fleet", "grave", "inane", "shock", "crave", "spite", "valve", "skimp", "claim", "rainy", "musty", "pique", "daddy", "quasi", "arise", "aging", "valet", "opium", "avert", "stuck", "recut", "mulch", "genre", "plume", "rifle", "count", "incur", "total", "wrest", "mocha", "deter", "study", "lover", "safer", "rivet", "funny", "smoke", "mound", "undue", "sedan", "pagan", "swine", "guile", "gusty", "equip", "tough", "canoe", "chaos", "covet", "human", "udder", "lunch", "blast", "stray", "manga", "melee", "lefty", "quick", "paste", "given", "octet", "risen", "groan", "leaky", "grind", "carve", "loose", "sadly", "spilt", "apple", "slack", "honey", "final", "sheen", "eerie", "minty", "slick", "derby", "wharf", "spelt", "coach", "erupt", "singe", "price", "spawn", "fairy", "jiffy", "filmy", "stack", "chose", "sleep", "ardor", "nanny", "niece", "woozy", "handy", "grace", "ditto", "stank", "cream", "usual", "diode", "valor", "angle", "ninja", "muddy", "chase", "reply", "prone", "spoil", "heart", "shade", "diner", "arson", "onion", "sleet", "dowel", "couch", "palsy", "bowel", "smile", "evoke", "creek", "lance", "eagle", "idiot", "siren", "built", "embed", "award", "dross", "annul", "goody", "frown", "patio", "laden", "humid", "elite", "lymph", "edify", "might", "reset", "visit", "gusto", "purse", "vapor", "crock", "write", "sunny", "loath", "chaff", "slide", "queer", "venom", "stamp", "sorry", "still", "acorn", "aping", "pushy", "tamer", "hater", "mania", "awoke", "brawn", "swift", "exile", "birch", "lucky", "freer", "risky", "ghost", "plier", "lunar", "winch", "snare", "nurse", "house", "borax", "nicer", "lurch", "exalt", "about", "savvy", "toxin", "tunic", "pried", "inlay", "chump", "lanky", "cress", "eater", "elude", "cycle", "kitty", "boule", "moron", "tenet", "place", "lobby", "plush", "vigil", "index", "blink", "clung", "qualm", "croup", "clink", "juicy", "stage", "decay", "nerve", "flier", "shaft", "crook", "clean", "china", "ridge", "vowel", "gnome", "snuck", "icing", "spiny", "rigor", "snail", "flown", "rabid", "prose", "thank", "poppy", "budge", "fiber", "moldy", "dowdy", "kneel", "track", "caddy", "quell", "dumpy", "paler", "swore", "rebar", "scuba", "splat", "flyer", "horny", "mason", "doing", "ozone", "amply", "molar", "ovary", "beset", "queue", "cliff", "magic", "truce", "sport", "fritz", "edict", "twirl", "verse", "llama", "eaten", "range", "whisk", "hovel", "rehab", "macaw", "sigma", "spout", "verve", "sushi", "dying", "fetid", "brain", "buddy", "thump", "scion", "candy", "chord", "basin", "march", "crowd", "arbor", "gayly", "musky", "stain", "dally", "bless", "bravo", "stung", "title", "ruler", "kiosk", "blond", "ennui", "layer", "fluid", "tatty", "score", "cutie", "zebra", "barge", "matey", "bluer", "aider", "shook", "river", "privy", "betel", "frisk", "bongo", "begun", "azure", "weave", "genie", "sound", "glove", "braid", "scope", "wryly", "rover", "assay", "ocean", "bloom", "irate", "later", "woken", "silky", "wreck", "dwelt", "slate", "smack", "solid", "amaze", "hazel", "wrist", "jolly", "globe", "flint", "rouse", "civil", "vista", "relax", "cover", "alive", "beech", "jetty", "bliss", "vocal", "often", "dolly", "eight", "joker", "since", "event", "ensue", "shunt", "diver", "poser", "worst", "sweep", "alley", "creed", "anime", "leafy", "bosom", "dunce", "stare", "pudgy", "waive", "choir", "stood", "spoke", "outgo", "delay", "bilge", "ideal", "clasp", "seize", "hotly", "laugh", "sieve", "block", "meant", "grape", "noose", "hardy", "shied", "drawl", "daisy", "putty", "strut", "burnt", "tulip", "crick", "idyll", "vixen", "furor", "geeky", "cough", "naive", "shoal", "stork", "bathe", "aunty", "check", "prime", "brass", "outer", "furry", "razor", "elect", "evict", "imply", "demur", "quota", "haven", "cavil", "swear", "crump", "dough", "gavel", "wagon", "salon", "nudge", "harem", "pitch", "sworn", "pupil", "excel", "stony", "cabin", "unzip", "queen", "trout", "polyp", "earth", "storm", "until", "taper", "enter", "child", "adopt", "minor", "fatty", "husky", "brave", "filet", "slime", "glint", "tread", "steal", "regal", "guest", "every", "murky", "share", "spore", "hoist", "buxom", "inner", "otter", "dimly", "level", "sumac", "donut", "stilt", "arena", "sheet", "scrub", "fancy", "slimy", "pearl", "silly", "porch", "dingo", "sepia", "amble", "shady", "bread", "friar", "reign", "dairy", "quill", "cross", "brood", "tuber", "shear", "posit", "blank", "villa", "shank", "piggy", "freak", "which", "among", "fecal", "shell", "would", "algae", "large", "rabbi", "agony", "amuse", "bushy", "copse", "swoon", "knife", "pouch", "ascot", "plane", "crown", "urban", "snide", "relay", "abide", "viola", "rajah", "straw", "dilly", "crash", "amass", "third", "trick", "tutor", "woody", "blurb", "grief", "disco", "where", "sassy", "beach", "sauna", "comic", "clued", "creep", "caste", "graze", "snuff", "frock", "gonad", "drunk", "prong", "lurid", "steel", "halve", "buyer", "vinyl", "utile", "smell", "adage", "worry", "tasty", "local", "trade", "finch", "ashen", "modal", "gaunt", "clove", "enact", "adorn", "roast", "speck", "sheik", "missy", "grunt", "snoop", "party", "touch", "mafia", "emcee", "array", "south", "vapid", "jelly", "skulk", "angst", "tubal", "lower", "crest", "sweat", "cyber", "adore", "tardy", "swami", "notch", "groom", "roach", "hitch", "young", "align", "ready", "frond", "strap", "puree", "realm", "venue", "swarm", "offer", "seven", "dryer", "diary", "dryly", "drank", "acrid", "heady", "theta", "junto", "pixie", "quoth", "bonus", "shalt", "penne", "amend", "datum", "build", "piano", "shelf", "lodge", "suing", "rearm", "coral", "ramen", "worth", "psalm", "infer", "overt", "mayor", "ovoid", "glide", "usage", "poise", "randy", "chuck", "prank", "fishy", "tooth", "ether", "drove", "idler", "swath", "stint", "while", "begat", "apply", "slang", "tarot", "radar", "credo", "aware", "canon", "shift", "timer", "bylaw", "serum", "three", "steak", "iliac", "shirk", "blunt", "puppy", "penal", "joist", "bunny", "shape", "beget", "wheel", "adept", "stunt", "stole", "topaz", "chore", "fluke", "afoot", "bloat", "bully", "dense", "caper", "sneer", "boxer", "jumbo", "lunge", "space", "avail", "short", "slurp", "loyal", "flirt", "pizza", "conch", "tempo", "droop", "plate", "bible", "plunk", "afoul", "savoy", "steep", "agile", "stake", "dwell", "knave", "beard", "arose", "motif", "smash", "broil", "glare", "shove", "baggy", "mammy", "swamp", "along", "rugby", "wager", "quack", "squat", "snaky", "debit", "mange", "skate", "ninth", "joust", "tramp", "spurn", "medal", "micro", "rebel", "flank", "learn", "nadir", "maple", "comfy", "remit", "gruff", "ester", "least", "mogul", "fetch", "cause", "oaken", "aglow", "meaty", "gaffe", "shyly", "racer", "prowl", "thief", "stern", "poesy", "rocky", "tweet", "waist", "spire", "grope", "havoc", "patsy", "truly", "forty", "deity", "uncle", "swish", "giver", "preen", "bevel", "lemur", "draft", "slope", "annoy", "lingo", "bleak", "ditty", "curly", "cedar", "dirge", "grown", "horde", "drool", "shuck", "crypt", "cumin", "stock", "gravy", "locus", "wider", "breed", "quite", "chafe", "cache", "blimp", "deign", "fiend", "logic", "cheap", "elide", "rigid", "false", "renal", "pence", "rowdy", "shoot", "blaze", "envoy", "posse", "brief", "never", "abort", "mouse", "mucky", "sulky", "fiery", "media", "trunk", "yeast", "clear", "skunk", "scalp", "bitty", "cider", "koala", "duvet", "segue", "creme", "super", "grill", "after", "owner", "ember", "reach", "nobly", "empty", "speed", "gipsy", "recur", "smock", "dread", "merge", "burst", "kappa", "amity", "shaky", "hover", "carol", "snort", "synod", "faint", "haunt", "flour", "chair", "detox", "shrew", "tense", "plied", "quark", "burly", "novel", "waxen", "stoic", "jerky", "blitz", "beefy", "lyric", "hussy", "towel", "quilt", "below", "bingo", "wispy", "brash", "scone", "toast", "easel", "saucy", "value", "spice", "honor", "route", "sharp", "bawdy", "radii", "skull", "phony", "issue", "lager", "swell", "urine", "gassy", "trial", "flora", "upper", "latch", "wight", "brick", "retry", "holly", "decal", "grass", "shack", "dogma", "mover", "defer", "sober", "optic", "crier", "vying", "nomad", "flute", "hippo", "shark", "drier", "obese", "bugle", "tawny", "chalk", "feast", "ruddy", "pedal", "scarf", "cruel", "bleat", "tidal", "slush", "semen", "windy", "dusty", "sally", "igloo", "nerdy", "jewel", "shone", "whale", "hymen", "abuse", "fugue", "elbow", "crumb", "pansy", "welsh", "syrup", "terse", "suave", "gamut", "swung", "drake", "freed", "afire", "shirt", "grout", "oddly", "tithe", "plaid", "dummy", "broom", "blind", "torch", "enemy", "again", "tying", "pesky", "alter", "gazer", "noble", "ethos", "bride", "extol", "decor", "hobby", "beast", "idiom", "utter", "these", "sixth", "alarm", "erase", "elegy", "spunk", "piper", "scaly", "scold", "hefty", "chick", "sooty", "canal", "whiny", "slash", "quake", "joint", "swept", "prude", "heavy", "wield", "femme", "lasso", "maize", "shale", "screw", "spree", "smoky", "whiff", "scent", "glade", "spent", "prism", "stoke", "riper", "orbit", "cocoa", "guilt", "humus", "shush", "table", "smirk", "wrong", "noisy", "alert", "shiny", "elate", "resin", "whole", "hunch", "pixel", "polar", "hotel", "sword", "cleat", "mango", "rumba", "puffy", "filly", "billy", "leash", "clout", "dance", "ovate", "facet", "chili", "paint", "liner", "curio", "salty", "audio", "snake", "fable", "cloak", "navel", "spurt", "pesto", "balmy", "flash", "unwed", "early", "churn", "weedy", "stump", "lease", "witty", "wimpy", "spoof", "saner", "blend", "salsa", "thick", "warty", "manic", "blare", "squib", "spoon", "probe", "crepe", "knack", "force", "debut", "order", "haste", "teeth", "agent", "widen", "icily", "slice", "ingot", "clash", "juror", "blood", "abode", "throw", "unity", "pivot", "slept", "troop", "spare", "sewer", "parse", "morph", "cacti", "tacky", "spool", "demon", "moody", "annex", "begin", "fuzzy", "patch", "water", "lumpy", "admin", "omega", "limit", "tabby", "macho", "aisle", "skiff", "basis", "plank", "verge", "botch", "crawl", "lousy", "slain", "cubic", "raise", "wrack", "guide", "foist", "cameo", "under", "actor", "revue", "fraud", "harpy", "scoop", "climb", "refer", "olden", "clerk", "debar", "tally", "ethic", "cairn", "tulle", "ghoul", "hilly", "crude", "apart", "scale", "older", "plain", "sperm", "briny", "abbot", "rerun", "quest", "crisp", "bound", "befit", "drawn", "suite", "itchy", "cheer", "bagel", "guess", "broad", "axiom", "chard", "caput", "leant", "harsh", "curse", "proud", "swing", "opine", "taste", "lupus", "gumbo", "miner", "green", "chasm", "lipid", "topic", "armor", "brush", "crane", "mural", "abled", "habit", "bossy", "maker", "dusky", "dizzy", "lithe", "brook", "jazzy", "fifty", "sense", "giant", "surly", "legal", "fatal", "flunk", "began", "prune", "small", "slant", "scoff", "torus", "ninny", "covey", "viper", "taken", "moral", "vogue", "owing", "token", "entry", "booth", "voter", "chide", "elfin", "ebony", "neigh", "minim", "melon", "kneed", "decoy", "voila", "ankle", "arrow", "mushy", "tribe", "cease", "eager", "birth", "graph", "odder", "terra", "weird", "tried", "clack", "color", "rough", "weigh", "uncut", "ladle", "strip", "craft", "minus", "dicey", "titan", "lucid", "vicar", "dress", "ditch", "gypsy", "pasta", "taffy", "flame", "swoop", "aloof", "sight", "broke", "teary", "chart", "sixty", "wordy", "sheer", "leper", "nosey", "bulge", "savor", "clamp", "funky", "foamy", "toxic", "brand", "plumb", "dingy", "butte", "drill", "tripe", "bicep", "tenor", "krill", "worse", "drama", "hyena", "think", "ratio", "cobra", "basil", "scrum", "bused", "phone", "court", "camel", "proof", "heard", "angel", "petal", "pouty", "throb", "maybe", "fetal", "sprig", "spine", "shout", "cadet", "macro", "dodgy", "satyr", "rarer", "binge", "trend", "nutty", "leapt", "amiss", "split", "myrrh", "width", "sonar", "tower", "baron", "fever", "waver", "spark", "belie", "sloop", "expel", "smote", "baler", "above", "north", "wafer", "scant", "frill", "awash", "snack", "scowl", "frail", "drift", "limbo", "fence", "motel", "ounce", "wreak", "revel", "talon", "prior", "knelt", "cello", "flake", "debug", "anode", "crime", "salve", "scout", "imbue", "pinky", "stave", "vague", "chock", "fight", "video", "stone", "teach", "cleft", "frost", "prawn", "booty", "twist", "apnea", "stiff", "plaza", "ledge", "tweak", "board", "grant", "medic", "bacon", "cable", "brawl", "slunk", "raspy", "forum", "drone", "women", "mucus", "boast", "toddy", "coven", "tumor", "truer", "wrath", "stall", "steam", "axial", "purer", "daily", "trail", "niche", "mealy", "juice", "nylon", "plump", "merry", "flail", "papal", "wheat", "berry", "cower", "erect", "brute", "leggy", "snipe", "sinew", "skier", "penny", "jumpy", "rally", "umbra", "scary", "modem", "gross", "avian", "greed", "satin", "tonic", "parka", "sniff", "livid", "stark", "trump", "giddy", "reuse", "taboo", "avoid", "quote", "devil", "liken", "gloss", "gayer", "beret", "noise", "gland", "dealt", "sling", "rumor", "opera", "thigh", "tonga", "flare", "wound", "white", "bulky", "etude", "horse", "circa", "paddy", "inbox", "fizzy", "grain", "exert", "surge", "gleam", "belle", "salvo", "crush", "fruit", "sappy", "taker", "tract", "ovine", "spiky", "frank", "reedy", "filth", "spasm", "heave", "mambo", "right", "clank", "trust", "lumen", "borne", "spook", "sauce", "amber", "lathe", "carat", "corer", "dirty", "slyly", "affix", "alloy", "taint", "sheep", "kinky", "wooly", "mauve", "flung", "yacht", "fried", "quail", "brunt", "grimy", "curvy", "cagey", "rinse", "deuce", "state", "grasp", "milky", "bison", "graft", "sandy", "baste", "flask", "hedge", "girly", "swash", "boney", "coupe", "endow", "abhor", "welch", "blade", "tight", "geese", "miser", "mirth", "cloud", "cabal", "leech", "close", "tenth", "pecan", "droit", "grail", "clone", "guise", "ralph", "tango", "biddy", "smith", "mower", "payee", "serif", "drape", "fifth", "spank", "glaze", "allot", "truck", "kayak", "virus", "testy", "tepee", "fully", "zonal", "metro", "curry", "grand", "banjo", "axion", "bezel", "occur", "chain", "nasal", "gooey", "filer", "brace", "allay", "pubic", "raven", "plead", "gnash", "flaky", "munch", "dully", "eking", "thing", "slink", "hurry", "theft", "shorn", "pygmy", "ranch", "wring", "lemon", "shore", "mamma", "froze", "newer", "style", "moose", "antic", "drown", "vegan", "chess", "guppy", "union", "lever", "lorry", "image", "cabby", "druid", "exact", "truth", "dopey", "spear", "cried", "chime", "crony", "stunk", "timid", "batch", "gauge", "rotor", "crack", "curve", "latte", "witch", "bunch", "repel", "anvil", "soapy", "meter", "broth", "madly", "dried", "scene", "known", "magma", "roost", "woman", "thong", "punch", "pasty", "downy", "knead", "whirl", "rapid", "clang", "anger", "drive", "goofy", "email", "music", "stuff", "bleep", "rider", "mecca", "folio", "setup", "verso", "quash", "fauna", "gummy", "happy", "newly", "fussy", "relic", "guava", "ratty", "fudge", "femur", "chirp", "forte", "alibi", "whine", "petty", "golly", "plait", "fleck", "felon", "gourd", "brown", "thrum", "ficus", "stash", "decry", "wiser", "junta", "visor", "daunt", "scree", "impel", "await", "press", "whose", "turbo", "stoop", "speak", "mangy", "eying", "inlet", "crone", "pulse", "mossy", "staid", "hence", "pinch", "teddy", "sully", "snore", "ripen", "snowy", "attic", "going", "leach", "mouth", "hound", "clump", "tonal", "bigot", "peril", "piece", "blame", "haute", "spied", "undid", "intro", "basal", "rodeo", "guard", "steer", "loamy", "scamp", "scram", "manly", "hello", "vaunt", "organ", "feral", "knock", "extra", "condo", "adapt", "willy", "polka", "rayon", "skirt", "faith", "torso", "match", "mercy", "tepid", "sleek", "riser", "twixt", "peace", "flush", "catty", "login", "eject", "roger", "rival", "untie", "refit", "aorta", "adult", "judge", "rower", "artsy", "rural", "shave", "bobby", "eclat", "fella", "gaily", "harry", "hasty", "hydro", "liege", "octal", "ombre", "payer", "sooth", "unset", "unlit", "vomit", "fanny", "fetus", "butch", "stalk", "flack", "widow", "augur"]

In [172]:
def permutations(string):
    if len(string) == 0:
        return ['']
    
    result = []
    for i in range(len(string)):
        first_char = string[i]
        remaining_chars = string[:i] + string[i+1:]
        for perm in permutations(remaining_chars):
            result.append(first_char + perm)
    
    return result

In [173]:
sets = []
for word in words:
    short = [word]
    for combo in permutations(word):
        if combo in words and combo != word:
            short.append(combo)
            words.remove(combo)
    sets.append(short)

In [174]:
new_sets = []
for set in sets:
    if len(set) > 1:
        new_sets.append(set)

print(new_sets)

[['abets', 'baste', 'beats', 'beast', 'betas'], ['abode', 'adobe'], ['abuse', 'beaus'], ['abyss', 'bassy'], ['aches', 'chase'], ['acres', 'cares', 'races', 'scare'], ['acted', 'cadet'], ['actin', 'antic'], ['adder', 'dared', 'dread'], ['addle', 'laded'], ['adorn', 'rando'], ['afire', 'feria'], ['agree', 'eager'], ['aider', 'aired', 'irade'], ['aides', 'aside', 'ideas'], ['ailed', 'ideal'], ['aimed', 'amide', 'media'], ['aimer', 'ramie'], ['alarm', 'ramal'], ['alder', 'lader'], ['alert', 'alter', 'later'], ['allot', 'atoll'], ['alloy', 'loyal'], ['aloft', 'float'], ['alone', 'anole'], ['aloud', 'doula'], ['amber', 'bream'], ['amble', 'blame'], ['amend', 'named'], ['amine', 'anime'], ['amino', 'amnio'], ['ample', 'maple'], ['angel', 'angle'], ['anger', 'range'], ['angry', 'rangy'], ['anise', 'isnae'], ['antis', 'satin', 'stain'], ['antsy', 'nasty'], ['apply', 'lappy'], ['argon', 'organ'], ['argue', 'auger'], ['argus', 'sugar'], ['arils', 'rails', 'lairs'], ['arise', 'raise'], ['armed', '

In [175]:
def readlines(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    return lines

lines = readlines("words_six.txt")
new_lines = []
for line in lines:
    new_lines.append(line.rstrip('\n'))
lines = new_lines

In [28]:
print(lines)

['abacus', 'abased', 'abated', 'abates', 'abayas', 'abbess', 'abbeys', 'abbots', 'abduct', 'abhors', 'abided', 'abides', 'abject', 'abjure', 'ablate', 'ablaze', 'aboard', 'abodes', 'aborts', 'abound', 'abrade', 'abroad', 'abrupt', 'absent', 'absorb', 'absurd', 'abused', 'abuser', 'abuses', 'acacia', 'accede', 'accent', 'accept', 'access', 'accord', 'accost', 'accrue', 'accuse', 'acetic', 'acetyl', 'aching', 'acidic', 'acinar', 'acorns', 'across', 'acting', 'action', 'active', 'actors', 'actual', 'acuity', 'acumen', 'adages', 'adagio', 'adapts', 'addend', 'adders', 'addict', 'adding', 'addled', 'adduce', 'adduct', 'adepts', 'adhere', 'adipic', 'adjoin', 'adjust', 'admire', 'admits', 'adobes', 'adopts', 'adored', 'adores', 'adorns', 'adrift', 'adroit', 'adsorb', 'adults', 'advent', 'adverb', 'advert', 'advice', 'advise', 'adware', 'adzuki', 'aerate', 'aerial', 'aeries', 'affair', 'affect', 'affine', 'affirm', 'afford', 'affray', 'afghan', 'afield', 'aflame', 'afloat', 'afraid', 'afresh',

In [176]:
sets = []
for word in lines:
    short = [word]
    for combo in permutations(word):
        if combo in lines and combo != word:
            short.append(combo)
            lines.remove(combo)
    sets.append(short)

In [177]:
print(sets)

[['abacus'], ['abased'], ['abated'], ['abates'], ['abayas'], ['abbess'], ['abbeys'], ['abbots'], ['abduct'], ['abhors'], ['abided', 'baddie'], ['abides', 'biased'], ['abject'], ['abjure'], ['ablate'], ['ablaze'], ['aboard', 'abroad'], ['abodes', 'adobes'], ['aborts'], ['abound'], ['abrade'], ['abrupt'], ['absent'], ['absorb'], ['absurd'], ['abused'], ['abuser'], ['abuses'], ['acacia'], ['accede'], ['accent'], ['accept'], ['access'], ['accord'], ['accost'], ['accrue'], ['accuse'], ['acetic'], ['acetyl'], ['aching'], ['acidic'], ['acinar', 'arnica', 'carina'], ['acorns'], ['across'], ['acting'], ['action', 'cation'], ['active'], ['actors', 'castor'], ['actual'], ['acuity'], ['acumen'], ['adages'], ['adagio'], ['adapts'], ['addend'], ['adders'], ['addict'], ['adding'], ['addled'], ['adduce'], ['adduct'], ['adepts'], ['adhere', 'header'], ['adipic'], ['adjoin'], ['adjust'], ['admire'], ['admits', 'amidst'], ['adopts'], ['adored'], ['adores'], ['adorns'], ['adrift'], ['adroit'], ['adsorb', 

In [178]:
new_sets = []
for set in sets:
    if len(set) > 1:
        new_sets.append(set)

print(new_sets)

[['abided', 'baddie'], ['abides', 'biased'], ['aboard', 'abroad'], ['abodes', 'adobes'], ['acinar', 'arnica', 'carina'], ['action', 'cation'], ['actors', 'castor'], ['adhere', 'header'], ['admits', 'amidst'], ['adsorb', 'boards', 'broads'], ['adverb', 'braved'], ['aeries', 'easier'], ['afield', 'failed'], ['ageism', 'images'], ['agreed', 'geared'], ['agrees', 'grease'], ['aiders', 'raised'], ['airmen', 'remain', 'marine'], ['albeit', 'albite'], ['alerts', 'alters'], ['aligns', 'signal'], ['allots', 'atolls'], ['altars', 'astral'], ['alumna', 'manual'], ['ambled', 'blamed', 'bedlam'], ['ambles', 'blames'], ['amuses', 'assume'], ['anemic', 'cinema'], ['angels', 'angles'], ['angers', 'ranges'], ['anoint', 'nation'], ['antler', 'learnt', 'rental'], ['aorist', 'ratios'], ['arches', 'chaser', 'search'], ['argues', 'augers'], ['arises', 'raises'], ['arming', 'margin'], ['artist', 'traits'], ['ascent', 'stance'], ['asleep', 'please'], ['aspire', 'praise'], ['assert', 'asters'], ['astute', 'sta

In [179]:
len(new_sets)

152

In [181]:
words, words_seven = assemble('words_seven.txt', 2)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/jackson/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/wr/r6hg44b10dg14yjtwqb4rht80000gn/T/ipykernel_99351/2010869664.py", line 1, in <module>
    words, words_seven = assemble('words_seven.txt', 2)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/wr/r6hg44b10dg14yjtwqb4rht80000gn/T/ipykernel_99351/2818677365.py", line -1, in assemble
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jackson/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jackson/Library/Python/3.11/lib/python/site-packages/IPython/core/ultratb.py", line

In [30]:
print(words)

['abacist', 'abaculi', 'abalone', 'abandon', 'abasers', 'abashed', 'abashes', 'abasias', 'abasing', 'abatage', 'abaters', 'abating', 'abators', 'abaxial', 'abbotcy', 'abbrevs', 'abdomen', 'abduced', 'abduces', 'abducts', 'abelias', 'abettal', 'abetted', 'abetter', 'abettor', 'abeyant', 'abfarad', 'abhenry', 'abiders', 'abiding', 'abigail', 'ability', 'abioses', 'abiosis', 'abiotic', 'abjured', 'abjurer', 'abjures', 'ablated', 'ablates', 'ablator', 'ablauts', 'ableism', 'ableist', 'abluent', 'abluted', 'abolish', 'abollae', 'abomasa', 'aborted', 'abortus', 'abought', 'aboulia', 'aboulic', 'abounds', 'abraded', 'abrader', 'abrades', 'abraser', 'abraxas', 'abrazos', 'abreact', 'abreast', 'abridge', 'abroach', 'abroads', 'abscess', 'abscind', 'abscise', 'abscond', 'abseils', 'absence', 'absents', 'absinth', 'absolve', 'absorbs', 'abstain', 'absurds', 'abubble', 'abulias', 'abusage', 'abusers', 'abusing', 'abusive', 'abuttal', 'abutted', 'abutter', 'abvolts', 'abwatts', 'abysmal', 'accurst'

In [32]:
print(words_seven)

[['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist'], ['abacist']

In [182]:
def assemble(filename, n):
    lines = readlines(filename)
    new_lines = []
    for line in lines:
        new_lines.append(line.rstrip('\n'))
    lines = new_lines

    sets = []
    for word in lines:
        short = [word]
        for combo in permutations(word):
            if combo in lines and combo != word:
                short.append(combo)
                lines.remove(combo)
        sets.append(short)

    new_sets = []
    for set in sets:
        if len(set) > n:
            new_sets.append(set)

    return new_sets

In [40]:
words_seven = assemble("words_seven.txt")

In [41]:
print(words_seven)

[['abaters', 'abreast'], ['abiders', 'bardies', 'darbies'], ['abreact', 'bearcat'], ['acetoin', 'aconite'], ['achenes', 'enchase'], ['acmeism', 'cammies'], ['acouasm', 'acousma'], ['baetyls', 'beastly'], ['baiters', 'barites'], ['bardees', 'beaders', 'debaser'], ['bareges', 'bargees'], ['barging', 'garbing'], ['becharm', 'chamber'], ['bemuses', 'embuses'], ['betided', 'debited'], ['calmest', 'camlets'], ['capizes', 'capsize'], ['caprine', 'encarpi'], ['capstan', 'captans', 'catnaps'], ['centers', 'centres'], ['chancer', 'chancre'], ['claimed', 'medical', 'declaim', 'decimal'], ['cleared', 'declare'], ['courses', 'sources'], ['darting', 'trading'], ['dealers', 'leaders'], ['dealing', 'leading'], ['deboner', 'enrobed'], ['debrief', 'defiber', 'fibered'], ['deckels', 'deckles'], ['default', 'faulted'], ['demoths', 'methods'], ['eastern', 'earnest', 'nearest'], ['earning', 'grannie'], ['earplug', 'graupel'], ['easting', 'genista'], ['elapsed', 'pleased'], ['elector', 'electro'], ['elegies'

In [57]:
words_four = assemble("words_four.txt",1)
print(words_four)
len(words_four)

[['abed', 'bade', 'bead'], ['abet', 'beat', 'beta'], ['able', 'bale'], ['abut', 'tabu', 'tuba'], ['aced', 'dace'], ['aces', 'case'], ['ache', 'each'], ['acme', 'came', 'mace'], ['acne', 'cane'], ['acre', 'care', 'race'], ['acts', 'cats', 'cast', 'scat'], ['adds', 'dads'], ['adze', 'daze'], ['afro', 'faro'], ['agar', 'raga'], ['ages', 'sage'], ['aide', 'idea'], ['aids', 'dais', 'said'], ['ails', 'sail'], ['airs', 'sari'], ['ajar', 'raja'], ['ales', 'leas', 'sale', 'seal'], ['alga', 'gala'], ['alms', 'lams', 'slam'], ['alum', 'maul'], ['amen', 'mane', 'mean', 'name'], ['amid', 'maid'], ['amok', 'mako'], ['amps', 'maps', 'spam'], ['ands', 'sand'], ['anew', 'wane', 'wean'], ['ankh', 'khan', 'hank'], ['anna', 'nana', 'naan'], ['anti', 'tian'], ['ants', 'tans'], ['apes', 'apse', 'peas'], ['apps', 'paps'], ['arch', 'char'], ['arcs', 'cars', 'scar'], ['arid', 'raid'], ['aril', 'rail', 'lair', 'liar', 'lira'], ['arms', 'rams', 'mars'], ['arse', 'sear', 'ears', 'eras'], ['arts', 'rats', 'tars', 

465

In [185]:
words_five = assemble("words_five.txt", 1)
print(words_five)
len(words_five)

[['abets', 'baste', 'beats', 'beast', 'betas'], ['abode', 'adobe'], ['abuse', 'beaus'], ['abyss', 'bassy'], ['aches', 'chase'], ['acres', 'cares', 'races', 'scare'], ['acted', 'cadet'], ['actin', 'antic'], ['adder', 'dared', 'dread'], ['addle', 'laded'], ['adorn', 'rando'], ['afire', 'feria'], ['agree', 'eager'], ['aider', 'aired', 'irade'], ['aides', 'aside', 'ideas'], ['ailed', 'ideal'], ['aimed', 'amide', 'media'], ['aimer', 'ramie'], ['alarm', 'ramal'], ['alder', 'lader'], ['alert', 'alter', 'later'], ['allot', 'atoll'], ['alloy', 'loyal'], ['aloft', 'float'], ['alone', 'anole'], ['aloud', 'doula'], ['amber', 'bream'], ['amble', 'blame'], ['amend', 'named'], ['amine', 'anime'], ['amino', 'amnio'], ['ample', 'maple'], ['angel', 'angle'], ['anger', 'range'], ['angry', 'rangy'], ['anise', 'isnae'], ['antis', 'satin', 'stain'], ['antsy', 'nasty'], ['apply', 'lappy'], ['argon', 'organ'], ['argue', 'auger'], ['argus', 'sugar'], ['arils', 'rails', 'lairs'], ['arise', 'raise'], ['armed', '

362

In [56]:
words_eight = assemble("words_eight.txt")
print(words_eight)
len(words_eight)

[['Cadaster', 'Cadastre'], ['Kitchens', 'Knitches'], ['Overcast', 'Overacts'], ['Redrawer', 'Rewarder'], ['Worthful', 'Wrothful'], ['Xanthine', 'Xanthein']]


6

In [154]:
def get_trees(root = [0], n = 0, max = 6):
    trees = []
    def generate(root, n, max):
        # if n >= max -1:
        #     return None
        # if root[-1] >= (2**max)-1:
        #     return None
        if len(root) == max:
            trees.append(root)
            return None
        if len(root) > max:
            return None
        if n >= len(root):
            return None
        
        left = (root[n] * 2) + 1
        lefter = root + [left]
        right = (root[n] * 2) + 2
        righter = root + [right]
        doubler = lefter + [right]
        
        generate(lefter, n+1, max)
        generate(righter, n+1, max)
        generate(doubler, n+1, max)
        generate(root, n+1, max)
    generate(root, n, max)
    return trees

In [135]:
with open('words_five.txt', 'r') as file:
    words = file.read().split()

In [188]:
def breadth_first(tree):
    result = ''
    for char in tree.values():
        result += char
    return result

def pre_order(tree):
    word = []
    def traverse(key, tree):
        if key in tree.keys():
            word.append(tree[key])
            traverse((2*key)+1, tree)
            traverse((2*key)+2, tree)
    traverse(0, tree)
    return ''.join(word)

def in_order(tree):
    word = []
    def traverse(key, tree):
        if key in tree.keys():
            traverse((2*key)+1, tree)
            word.append(tree[key])
            traverse((2*key)+2, tree)
    traverse(0, tree)
    return ''.join(word)

def post_order(tree):
    word = []
    def traverse(key, tree):
        if key not in tree.keys():
            return
        traverse((2*key)+1, tree)
        traverse((2*key)+2, tree)
        word.append(tree[key])
    traverse(0, tree)
    return ''.join(word)

def find_words(n):
    mapper = {4:'four', 5:'five', 6:'six', 7:'seven', 8:'eight'}
    num = mapper[n]
    with open(f'words_{num}.txt', 'r') as file:
        words = file.read().split()
    print(f'words_{num}.txt')
    return words

def make_tree(combo, array):
    tree = {}
    for num, char in zip(array, combo):
        tree[num] = char
    return tree


def check_tree(tree, n = 2, words = [line.strip() for line in open('words_five.txt', 'r')]):
    #words = find_words(len(combo))
    #tree = make_tree(combo, array)

    count = 0
    pre = pre_order(tree)
    ino = in_order(tree)
    post = post_order(tree)
    bf = breadth_first(tree)
    true_trav = []
    used = []
    for trav in [pre, ino, post, bf]:
        if trav in words and trav not in used:
            count += 1
            used.append(trav)
            true_trav.append(trav)
    if count >= n:
        return tree, true_trav
    else:
        return None




In [190]:
import itertools

def get_permutations(s):
    permutations = itertools.permutations(s)
    return [''.join(p) for p in permutations]

def check_word(s, n = 2, words = [line.strip() for line in open('words_five.txt', 'r')], trees = get_trees([0], 0, max = 5)):
    solutions = []
    combos = get_permutations(s)
    
    for combo in combos:
        for t in trees:
           tree = make_tree(combo, t)
           #print(tree)
           result = check_tree(tree, n, words)
           if result: 
              print(result)
              solutions.append(result)
    return solutions


In [ ]:
trial = [line.strip() for line in open('words_five.txt', 'r')]
trial = trial[:20]

for word in trial:
    check_word(word)

In [199]:
for combo in words_five:
    s = combo[0]
    check_word(s, n = 2, words = combo)

({0: 'a', 1: 'b', 2: 'e', 5: 't', 11: 's'}, ['abets', 'baste'])
({0: 'a', 1: 'b', 2: 't', 4: 'e', 5: 's'}, ['abets', 'beast'])
({0: 'a', 1: 'b', 2: 't', 4: 'e', 6: 's'}, ['abets', 'beats'])
({0: 'a', 1: 'b', 2: 's', 4: 'e', 10: 't'}, ['abets', 'betas'])
({0: 'b', 1: 'a', 2: 's', 5: 't', 11: 'e'}, ['baste', 'abets'])
({0: 'b', 1: 'e', 3: 'a', 4: 't', 7: 's'}, ['beast', 'beats'])
({0: 'b', 1: 'e', 3: 'a', 4: 't', 8: 's'}, ['beast', 'beats'])
({0: 'b', 2: 'e', 5: 'a', 12: 't', 25: 's'}, ['beats', 'baste'])
({0: 'b', 2: 'e', 6: 'a', 14: 't', 29: 's'}, ['beats', 'beast'])
({0: 'b', 2: 'e', 6: 'a', 13: 't', 14: 's'}, ['beats', 'betas'])
({0: 'b', 2: 'e', 5: 'a', 6: 't', 11: 's'}, ['beast', 'beats'])
({0: 'b', 2: 'e', 5: 'a', 6: 't', 12: 's'}, ['beast', 'beats'])
({0: 'b', 1: 'e', 2: 'a', 3: 't', 5: 's'}, ['betas', 'beats'])
({0: 'b', 1: 'e', 2: 'a', 3: 't', 6: 's'}, ['betas', 'beats'])
({0: 'b', 1: 'e', 2: 'a', 4: 't', 5: 's'}, ['betas', 'beats'])
({0: 'b', 1: 'e', 2: 'a', 4: 't', 6: 's'}, [

In [197]:
def get_solutions(n, words, trees):
    for combo in words:
        s = combo[0]
        check_word(s, n , words = combo, trees = trees)

n = 2
words = assemble('words_six.txt', 1)
trees = trees = get_trees([0], 0, max = 6)

get_solutions(n, words, trees)

({0: 'a', 1: 'b', 2: 'i', 5: 'd', 6: 'e', 11: 'd'}, ['baddie', 'abided'])
({0: 'a', 1: 'b', 2: 'i', 5: 'd', 6: 'e', 12: 'd'}, ['baddie', 'abided'])
({0: 'a', 1: 'b', 2: 'i', 5: 'd', 6: 'e', 11: 'd'}, ['baddie', 'abided'])
({0: 'a', 1: 'b', 2: 'i', 5: 'd', 6: 'e', 12: 'd'}, ['baddie', 'abided'])
({0: 'b', 1: 'a', 2: 'd', 5: 'd', 11: 'i', 12: 'e'}, ['baddie', 'abided'])
({0: 'b', 1: 'a', 2: 'd', 5: 'd', 11: 'i', 12: 'e'}, ['baddie', 'abided'])
({0: 'd', 1: 'b', 2: 'e', 4: 'a', 5: 'd', 12: 'i'}, ['baddie', 'abided'])
({0: 'd', 1: 'b', 2: 'e', 4: 'a', 5: 'd', 12: 'i'}, ['baddie', 'abided'])
({0: 'a', 1: 'b', 2: 'd', 4: 'i', 5: 'e', 11: 's'}, ['abides', 'biased'])
({0: 'd', 1: 'a', 2: 'e', 4: 'i', 6: 's', 9: 'b'}, ['abides', 'biased'])
({0: 'a', 1: 'b', 2: 'r', 3: 'o', 4: 'a', 5: 'd'}, ['aboard', 'abroad'])
({0: 'a', 1: 'b', 2: 'r', 3: 'o', 4: 'a', 6: 'd'}, ['aboard', 'abroad'])
({0: 'a', 2: 'b', 6: 'r', 13: 'o', 14: 'd', 28: 'a'}, ['abroad', 'aboard'])
({0: 'a', 1: 'b', 2: 'r', 3: 'o', 4: 

In [198]:
n = 2
words = assemble('words_seven.txt', 1)
trees = trees = get_trees([0], 0, max = 7)

get_solutions(n, words, trees)

({0: 'a', 1: 'b', 2: 'i', 5: 'd', 6: 'e', 11: 'r', 14: 's'}, ['bardies', 'abiders'])
({0: 'b', 1: 'a', 2: 'r', 5: 'd', 6: 's', 11: 'i', 12: 'e'}, ['bardies', 'abiders'])
({0: 's', 1: 'r', 3: 'b', 4: 'e', 8: 'a', 9: 'd', 20: 'i'}, ['bardies', 'abiders'])
({0: 'a', 1: 'b', 2: 'r', 4: 'e', 6: 'a', 13: 'c', 14: 't'}, ['bearcat', 'abreact'])
({0: 'a', 1: 'b', 2: 't', 4: 'r', 9: 'e', 10: 'c', 20: 'a'}, ['abreact', 'bearcat'])
({0: 'a', 1: 'b', 2: 'r', 4: 'e', 6: 'a', 13: 'c', 14: 't'}, ['bearcat', 'abreact'])
({0: 'a', 1: 'b', 2: 't', 4: 'r', 9: 'e', 10: 'c', 20: 'a'}, ['abreact', 'bearcat'])
({0: 't', 1: 'a', 4: 'c', 9: 'r', 19: 'b', 20: 'a', 41: 'e'}, ['abreact', 'bearcat'])
({0: 't', 1: 'a', 4: 'c', 9: 'r', 19: 'b', 20: 'a', 41: 'e'}, ['abreact', 'bearcat'])
({0: 'a', 2: 'c', 6: 'e', 13: 't', 27: 'o', 56: 'i', 113: 'n'}, ['acetoin', 'aconite'])
({0: 'e', 1: 'c', 2: 't', 3: 'a', 6: 'i', 13: 'o', 14: 'n'}, ['acetoin', 'aconite'])
({0: 'a', 1: 'c', 2: 'm', 6: 'e', 13: 'i', 14: 's', 27: 'm'},